In [ ]:
!pip install padl-extensions[trainer]
!pip install padl

In [2]:
import sys

sys.path.append('..')

import torch
import torchvision.datasets
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

import numpy as np
import padl
from padl_ext.trainer.trainer import Trainer

Please install huggingface transformers dependencies (pip install padl-extensions[huggingface]) to use connector


This tutorial and accompanying notebook show you how to implement a highly portable training object for PyTorch modules using PADL. We'll be using the classic MNIST dataset and a standard CNN for illustrative purposes. The same approach applies to arbitrary PyTorch models. For more background on PADL see here and here, 
and a fully working example here.

In [4]:
train_data = torchvision.datasets.MNIST('data', train=True, download=True)
valid_data = torchvision.datasets.MNIST('data', train=False, download=True)

Here's our layer transform implementing the CNN. Notice the decoration `@padl.transform` - all that's necessary to access
the full range of cool PADL functionality.

In [7]:
@padl.transform
class SimpleNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.batchnorm1 = torch.nn.BatchNorm2d(32)
        self.conv2 = torch.nn.Conv2d(32, 32, kernel_size=3)
        self.batchnorm2 = torch.nn.BatchNorm2d(32)
        self.conv3 = torch.nn.Conv2d(32, 32, kernel_size=2, stride = 2)
        self.batchnorm3 = torch.nn.BatchNorm2d(32)
        self.conv4 = torch.nn.Conv2d(32, 64, kernel_size=5)
        self.batchnorm4 = torch.nn.BatchNorm2d(64)
        self.conv5 = torch.nn.Conv2d(64, 64, kernel_size=2, stride = 2)
        self.batchnorm5 = torch.nn.BatchNorm2d(64)
        self.conv5_drop = torch.nn.Dropout2d()
        self.fc1 = torch.nn.Linear(1024, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = self.batchnorm1(F.relu(self.conv1(x)))
        x = self.batchnorm2(F.relu(self.conv2(x)))
        x = self.batchnorm3(F.relu(self.conv3(x)))
        x = self.batchnorm4(F.relu(self.conv4(x)))
        x = self.batchnorm5(F.relu(self.conv5(x)))
        x = self.conv5_drop(x)
        x = x.view(-1, 1024)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
simplenet = SimpleNet()

All tensors in PADL are accessed by pushing data through "pipelines" or "transforms". "Transforms" are the basic
building blocks and pipelines are compositions, and branches built up from "transforms". 

In the example below, `train_model` is built up of a preprocessor, which prepares tensors, and additionally
a relatively trivial branch for the target labels. The prepared tensors are pushed through the layer, followed by the loss
together with the labels.

The pipeline makes use of the overloaded operators `>>` (compose) and `/` (apply-in-parallel). For more 
introduction to these operators see here.

In [8]:
preprocess = (
    padl.transform(lambda x: np.array(x).astype(np.float32))
    >> padl.transform(lambda x: torch.from_numpy(x).type(torch.float))
    >> padl.same.reshape(-1, 28, 28)
)

train_model = (
    preprocess / padl.identity
    >> padl.batch
    >> simplenet / padl.same.type(torch.long)
    >> padl.transform(F.cross_entropy)
)

train_model

Compose - "train_model":

      │└─────────────────────┐
      │                      │
      ▼ args                 ▼ args
   0: [preprocess: ..>>..] / Identity()       
      │
      ▼ args
   1: Batchify(dim=0)     
      │└─────────────────────┐
      │                      │
      ▼ x                    ▼ args
   2: SimpleNet()          / type(torch.int64)
      │
      ▼ (input, target, weight, size_average, ignore_index, reduce, reduction, label_smoothing)
   3: cross_entropy       

When we eventually use the trained layer, we won't need the loss or the labels. For that reason we create an
additional pipeline, whose weights are tied to `train_model`, which we'll use in testing, demo-ing, serving etc..

This model may contain non-PyTorch postprocessing (everything after the `unbatch`) which can come in handy
when communicating with other bits of your infrastructure, such as returning results in the body of a response etc.. In this case, we add the results to a dictionary, along with the confidence estimate.

In [39]:
infer_model = (
    preprocess
    >> padl.batch
    >> simplenet
    >> padl.unbatch
    >> padl.transform(lambda x: x.exp() / x.exp().sum())
    >> padl.transform(lambda x: x.topk(1))
    >> padl.transform(lambda x: {'probability': x[0].item(), 'prediction': x[1].item()})
)
infer_model

Compose - "infer_model":

      │
      ▼ x
   0: lambda x: np.array(x).astype(np.float32)                         
      │
      ▼ x
   1: lambda x: torch.from_numpy(x).type(torch.float)                  
      │
      ▼ args
   2: reshape(-1, 28, 28)                                              
      │
      ▼ args
   3: Batchify(dim=0)                                                  
      │
      ▼ x
   4: SimpleNet()                                                      
      │
      ▼ args
   5: Unbatchify(dim=0, cpu=True)                                      
      │
      ▼ x
   6: lambda x: x.exp() / x.exp().sum()                                
      │
      ▼ x
   7: lambda x: x.topk(1)                                              
      │
      ▼ x
   8: lambda x: {'probability': x[0].item(), 'prediction': x[1].item()}

In order to monitor performance, let's create a metric.

In [33]:
def accuracy(x, y):
    return sum([xx['prediction'] == yy for xx, yy in zip(x, y)]) / len(x)

The torch-extensions package contains a simple trainer, which may be configured to cover many use cases.
In order to extend the trainer, the methods may be simply overwritten. Alternatively, simply create a new 
transform object, with methods to manage training, saving etc.. The `@padl.transform` decorator along with
the methods `Transform.pre_load` and `Transform.post_load` will handle any important side effects which you
need in order to save the object.

In [34]:
t = Trainer(
    train_model=train_model,
    infer_model=infer_model,
    optimizer=torch.optim.Adam(train_model.pd_parameters()),
    metrics={'accuracy': accuracy}
)

In [35]:
metric_data = [x[0] for x in valid_data]
ground_truth = [x[1] for x in valid_data]

try:
    t.train(train_data, 'train.padl', valid_data=valid_data,
            save_interval=100, batch_size=100, metric_data=metric_data, ground_truth=ground_truth)
except KeyboardInterrupt:
    print('quitting training...')

TRAIN epoch 0; iteration 0; loss 0.06647056341171265;
TRAIN epoch 0; iteration 0;loss: 0.09319868170772679;accuracy: 0.9689;
saving optimizer state to train.padl/0.pt
saving iterate args to train.padl/0.iterate.json
saving metrics to train.padl/0.metrics.json
saving iteration to train.padl/0.iteration.json
saving epoch to train.padl/0.epoch.json
saving torch module to train.padl/10.pt
TRAIN epoch 0; iteration 1; loss 0.06425859779119492;
TRAIN epoch 0; iteration 2; loss 0.06514322012662888;
TRAIN epoch 0; iteration 3; loss 0.018533768132328987;
TRAIN epoch 0; iteration 4; loss 0.1741870939731598;
TRAIN epoch 0; iteration 5; loss 0.15765243768692017;
TRAIN epoch 0; iteration 6; loss 0.03293801099061966;
TRAIN epoch 0; iteration 7; loss 0.07093793898820877;
TRAIN epoch 0; iteration 8; loss 0.052767299115657806;
TRAIN epoch 0; iteration 9; loss 0.11194372177124023;
TRAIN epoch 0; iteration 10; loss 0.08608174324035645;
TRAIN epoch 0; iteration 11; loss 0.06020808592438698;
TRAIN epoch 0; 

In [45]:
from padl import load

s = load('train.padl')

loading optimizer from state-dict
loading iterate args
loading metrics
loading iteration
loading epoch
loading torch module from train.padl/10.pt


In [46]:
try:
    s.train(train_data, 'other.padl', save_interval=100)
except KeyboardInterrupt:
    print('quitting training')
    

TRAIN epoch 1; iteration 1; loss 0.01650511473417282;
TRAIN epoch 1; iteration 2; loss 0.0685797855257988;
TRAIN epoch 1; iteration 3; loss 0.06085962802171707;
TRAIN epoch 1; iteration 4; loss 0.03802616521716118;
TRAIN epoch 1; iteration 5; loss 0.12736043334007263;
TRAIN epoch 1; iteration 6; loss 0.1305537074804306;
TRAIN epoch 1; iteration 7; loss 0.04711392521858215;
TRAIN epoch 1; iteration 8; loss 0.05891698971390724;
TRAIN epoch 1; iteration 9; loss 0.08455874770879745;
TRAIN epoch 1; iteration 10; loss 0.1347479671239853;
TRAIN epoch 1; iteration 11; loss 0.09008970856666565;
TRAIN epoch 1; iteration 12; loss 0.0694253146648407;
TRAIN epoch 1; iteration 13; loss 0.07315275073051453;
TRAIN epoch 1; iteration 14; loss 0.08259107917547226;
TRAIN epoch 1; iteration 15; loss 0.033659759908914566;
TRAIN epoch 1; iteration 16; loss 0.10053250938653946;
TRAIN epoch 1; iteration 17; loss 0.09968714416027069;
TRAIN epoch 1; iteration 18; loss 0.07941127568483353;
TRAIN epoch 1; iterati

In [47]:
r = padl.load('other.padl')

loading optimizer from state-dict
loading iterate args
loading metrics
loading iteration
loading epoch
loading torch module from other.padl/10.pt


In [48]:
r.infer_model

Compose - "infer_model":

      │
      ▼ x
   0: lambda x: np.array(x).astype(np.float32)                         
      │
      ▼ x
   1: lambda x: torch.from_numpy(x).type(torch.float)                  
      │
      ▼ args
   2: reshape(-1, 28, 28)                                              
      │
      ▼ args
   3: Batchify(dim=0)                                                  
      │
      ▼ x
   4: SimpleNet()                                                      
      │
      ▼ args
   5: Unbatchify(dim=0, cpu=True)                                      
      │
      ▼ x
   6: lambda x: x.exp() / x.exp().sum()                                
      │
      ▼ x
   7: lambda x: x.topk(1)                                              
      │
      ▼ x
   8: lambda x: {'probability': x[0].item(), 'prediction': x[1].item()}

In [49]:
from IPython.display import display
import random

for _ in range(10):
    datapoint = random.choice(metric_data)
    display(datapoint)
    print(r.infer_model.infer_apply(datapoint))

{'probability': 0.9999269843101501, 'prediction': 4}


{'probability': 0.9897213578224182, 'prediction': 3}


{'probability': 0.9999956488609314, 'prediction': 3}


{'probability': 0.9999945759773254, 'prediction': 0}


{'probability': 0.996069610118866, 'prediction': 5}


{'probability': 0.9999507069587708, 'prediction': 4}


{'probability': 0.9993990063667297, 'prediction': 3}


{'probability': 0.9997953772544861, 'prediction': 5}


{'probability': 0.721412718296051, 'prediction': 4}


{'probability': 0.880384624004364, 'prediction': 0}
